In [2]:
import geopandas as gpd
from shapely.ops import transform
from shapely.geometry import Point, Polygon #convert las to gpd
import rioxarray
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import pyproj
#import pygeos

In [23]:
import os
import json
import tempfile
import shutil
import re
import copy
from glob import glob
import pandas as pd
import requests
import volume_estimation_functions as vol_est


In [5]:
tank_data = gpd.read_file("//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk/complete_dataset_/"
                          "tile_level_annotations/tile_level_annotations.geojson")

In [24]:
tank_idx = 32
row = tank_data.iloc[tank_idx]

In [25]:
tnm_names = []
urls = []
geometries = []
tank_idxs = []

minx = row['nw_corner_polygon_lon']
miny = row['se_corner_polygon_lat']
maxx = row['se_corner_polygon_lon']
maxy = row['nw_corner_polygon_lat']
tank_polygon = Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny)])
tank_bbox = [minx, miny, maxx, maxy]
tank_bbox_str = ",".join(map(str, tank_bbox)) # https://stackoverflow.com/questions/32313623/passing-comma-separated-values-in-request-get-python

tnm_names_itr = copy.copy(tnm_names)
urls_itr = copy.copy(urls)
geometries_itr = copy.copy(geometries)

In [26]:
dataset_name="Digital Elevation Model (DEM) 1 meter"
timeout=120
request_total_idx='total'
request_content_idx='items'
request_content_names_idx='title'
tnm_url="https://tnmaccess.nationalmap.gov/api/v1/products"
#check if tank polygon is inside of a previously identified dataset
## https://stackoverflow.com/questions/4406389/if-else-in-a-list-comprehension
matching_poly=[(tnm_name, url, tnm_poly) for tnm_name, url, tnm_poly in zip(tnm_names_itr, urls_itr, geometries_itr) if tnm_poly.contains(tank_polygon)]
if len(matching_poly) > 0: #if it is inside of another dataset, add to the lists tracking names, urls, geoms, idx
    tnm_name, url, tnm_poly = matching_poly[0]
    tnm_names.append(tnm_name)
    urls.append(url)
    geometries.append(tnm_poly)
    tank_idxs.append(tank_idx)
else:
    payload = {'bbox': tank_bbox_str, "datasets": dataset_name}
    bool_request, contents_json = vol_est.submit_http_api_request(payload, tnm_url, timeout)
    print(bool_request, contents_json)
    if bool_request: #if the request was successful
        print(len(contents_json[request_content_idx]))
        bool_entries = vol_est.verify_entries_exist(contents_json, request_content_idx)
        print(bool_entries)
        if bool_entries: #if the request provided enteries
            contents_df = vol_est.json_to_dict(contents_json, request_content_idx, request_content_names_idx)
            tnm_names, urls, geometries = vol_est.get_dataset_of_interest(contents_df, tnm_names, urls, geometries)
            tank_idxs.append(tank_idx)

complete_df = pd.DataFrame(list(zip(tank_idxs, tnm_names, urls, geometries)),
                       index=tank_idxs, columns=['tank_idxs', 'usgs_tnm_names', 'urls', 'geometry'])


JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [35]:
maxy

32.65046580759526

In [29]:
r = requests.get(tnm_url, payload, timeout=timeout)
r.url

'https://tnmaccess.nationalmap.gov/api/v1/products?bbox=-102.65523622921305%2C32.6504047666544%2C-102.65515468130397%2C32.65046580759526&datasets=Digital+Elevation+Model+%28DEM%29+1+meter'

In [13]:
vol_est.submit_http_api_request(payload, tnm_url, timeout)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [16]:
tnm_url
timeout
payload

{'bbox': '-102.65523622921305,32.6504047666544,-102.65515468130397,32.65046580759526',
 'datasets': 'Digital Elevation Model (DEM) 1 meter'}

In [48]:
contents_json

{'total': 2,
 'items': [{'title': 'USGS one meter x60y450 NY CMPG 2013',
   'moreInfo': 'This is a tile of the standard one-meter resolution digital elevation model (DEM) produced through the 3D Elevation Program (3DEP). The elevations in this DEM represent the topographic bare-earth surface. USGS standard one-meter DEMs are produced exclusively from high resolution light detection and ranging (lidar) source data of one-meter or higher resolution. One-meter DEM surfaces are seamless within collection projects, but, not necessarily seamless across projects. The spatial reference used for tiles of the one-meter DEM within the conterminous United States (CONUS) is Universal Transverse Mercator (UTM) in units of meters, and in conformance with the North American Datum of 1983 (NAD83). All bare earth elevation values are in [...]',
   'sourceId': '5eacfdfe82cefae35a25180c',
   'sourceName': 'ScienceBase',
   'sourceOriginId': None,
   'sourceOriginName': 'gda',
   'metaUrl': 'https://www.sc